In [45]:
import re
import string
import nltk
import regex as re 
import pandas as pd 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

In [46]:
file_path = r"C:\Users\Sujal Karmakar\Desktop\Desktop\MyCareer\Data World\Data Analyst\Python\python_data_analytics_project\Theme Finder Using Caption (NLP)\Data\reviews_of_orders_during_crisis.xlsx"

df = pd.read_excel(file_path)

#### 1) Basic Cleaning (lower case, remover numbers, emoji, etc)

In [37]:
df_caption = df[["order_id", "review"]]

In [47]:
import re

# list of generic words and hashtags that i wanna remove
generic_words = set(
    [
        "the",
        "and",
        "to",
        "of",
        "in",
        "for",
        "this",
        "is",
        "you",
        "our",
        "by",
        "foundmademodern",
        "link",
        "bio",
        "we",
        "my",
        "from",
        "with",
        "was",
        "at",
        "followfriday",
        "please",
        "love",
        "follow",
        "introduce",
        "your",
        "on",
        "is",
        "that",
        "are",
        "all",
        "out",
        "more",
        "handmade",
        "it", 
        "art", 
        "diy", 
        "make", 
        "so", 
        "craftsposure", 
        "craft", 
        "be", 
        "shop", 
        "have"
    ]
)


def clean_caption(text):
    if pd.isna(text):
        return ""
    text = str(text).strip()
    text_lower = text.lower()

    text_lower = re.sub(r"#(\w+)", r"\1", text_lower)

    text_lower = re.sub(r"\s+", " ", text_lower)

    for phrase in generic_words:
        # Add word boundaries \b to avoid partial matches
        pattern = r"\b" + re.escape(phrase) + r"\b"
        text_lower = re.sub(pattern, "", text_lower, flags=re.IGNORECASE)

    text_lower = re.sub(r"\s+", " ", text_lower).strip()

    return text_lower


df_caption_cleaned = df[["order_id", "review"]].copy()

df_caption_cleaned["review_cleaned"] = df_caption_cleaned["review"].apply(
    clean_caption
)

print(df_caption_cleaned.head())

          order_id               review      review_cleaned
0  ORD202504000898  Not worth the price     not worth price
1  ORD202504020679  Not worth the price     not worth price
2  ORD202506000002  Not worth the price     not worth price
3  ORD202506000005   Average experience  average experience
4  ORD202506000006            Bad taste           bad taste


In [48]:
df_caption_cleaned.sample(15)

,order_id,review,review_cleaned
4794,ORD202507007526,Cold food,cold food
5254,ORD202507009373,Stale food served,stale food served
9417,ORD202509002663,Portion size smaller than expected,portion size smaller than expected
11575,ORD202509009365,Cold food,cold food
9949,ORD202509004264,Bad taste,bad taste
2983,ORD202506010466,Packaging was poor,packaging poor
11442,ORD202509008968,Food safety issue,food safety issue
10599,ORD202509006285,Packaging was poor,packaging poor
10970,ORD202509007452,Cold food,cold food
6260,ORD202508002496,Packaging was poor,packaging poor


#### 2) Tokenization (removing stop words)

In [40]:
# import nltk

# # Define a custom download directory
# custom_nltk_path = r'C:\Users\Sujal Karmakar\.conda\envs\DS\nltk_data'

# # Download resources to the custom path
## nltk.download('punkt', download_dir=custom_nltk_path)
# nltk.download('punkt_tab', download_dir=custom_nltk_path)
# nltk.download('stopwords', download_dir=custom_nltk_path)

# # Manually add the path to NLTK's search locations
# nltk.data.path.append(custom_nltk_path)

# i had to explicitly download these by specifying files and also punkt was not working for me nltk was demanding punkt_tab so i did that 


#### 3) Creating dictionaries and corpus for LDA (Linear discriminant analysis)

In [49]:
from bertopic import BERTopic

documents = df_caption_cleaned["review_cleaned"].tolist()

topic_model = BERTopic(language="english", nr_topics=20, verbose=True)

topics, probs = topic_model.fit_transform(documents)

topic_info = topic_model.get_topic_info()
print(topic_info)


for topic_num in topic_info["Topic"]:
    if topic_num != -1:  # -1 is the noise topic
        print(f"\nTopic {topic_num}:")
        print(topic_model.get_topic(topic_num))

2025-11-09 13:42:12,793 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/372 [00:00<?, ?it/s]

2025-11-09 13:42:23,885 - BERTopic - Embedding - Completed ✓
2025-11-09 13:42:23,886 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-09 13:42:26,052 - BERTopic - Dimensionality - Completed ✓
2025-11-09 13:42:26,053 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-09 13:42:26,498 - BERTopic - Cluster - Completed ✓
2025-11-09 13:42:26,499 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-11-09 13:42:26,551 - BERTopic - Representation - Completed ✓
2025-11-09 13:42:26,552 - BERTopic - Topic reduction - Reducing number of topics
2025-11-09 13:42:26,593 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-09 13:42:26,620 - BERTopic - Representation - Completed ✓
2025-11-09 13:42:26,623 - BERTopic - Topic reduction - Reduced number of topics from 86 to 20


    Topic  Count                         Name  \
0      -1    494     -1_good_quality_not_food   
1       0   1080      0_good_quality_not_food   
2       1   1076         1_safety_issue_food_   
3       2    997          2_packaging_issue__   
4       3    972                3_taste_bad__   
5       4    971         4_served_stale_food_   
6       5    942          5_recommended_not__   
7       6    904                6_very_late__   
8       7    668                7_cold_food__   
9       8    658  8_than_smaller_size_portion   
10      9    657           9_price_worth_not_   
11     10    645      10_experience_average__   
12     11    630          11_poor_packaging__   
13     12    630    12_great_quality_not_food   
14     13    156             13_worst_order__   
15     14    133             14_never_again__   
16     15    131        15_terrible_hygiene__   
17     16    127        16_service_horrible__   
18     17     11     17_good_quality_not_food   
19     18     11    

In [50]:
with open("bertopic_topics_2.txt", "w", encoding="utf-8") as f:
    for idx, topic in topic_model.get_topic_info().iterrows():
        f.write(f"Topic {topic['Topic']} (Count: {topic['Count']}):\n")
        terms = topic_model.get_topic(topic["Topic"])
        for term, weight in terms:
            f.write(f"  {term}: {weight:.4f}\n")
        f.write("\n")

In [51]:
# Mapping topic IDs to clear, professional names

# topic_labels = {
#     -1: "Bad Packaging",
#     0: "Quality(Stale/Cold/Hygiene)",
#     1: "Quality(Stale/Cold/Hygiene)",
#     2: "Bad Taste",
#     3: "Bad Service",
#     4: "Delivery Delay",
#     5: "Bad Packaging",
#     6: "Quality(Stale/Cold/Hygiene)",
#     7: "Smaller Quantity",
#     8: "High Price",
#     9: "Bad Service",
#     10: "Bad Packaging",
#     11: "Quality(Stale/Cold/Hygiene)",
#     12: "Quality(Stale/Cold/Hygiene)",
#     13: "Bad Service",
#     14: "Bad Service",
#     15: "Quality(Stale/Cold/Hygiene)",
#     16: "Bad Service",
#     17: "Quality(Stale/Cold/Hygiene)",
#     18: "Quality(Stale/Cold/Hygiene)",
# }
topic_labels = {
    -1: "Quality(Stale/Cold/Hygiene)",
    0: "Quality(Stale/Cold/Hygiene)",
    1: "Quality(Stale/Cold/Hygiene)",
    2: "Bad Packaging",
    3: "Bad Taste",
    4: "Quality(Stale/Cold/Hygiene)",
    5: "Bad Service",
    6: "Delivery Delay",
    7: "Quality(Stale/Cold/Hygiene)",
    8: "Smaller Quantity",
    9: "High Price",
    10: "Bad Service",
    11: "Bad Packaging",
    12: "Quality(Stale/Cold/Hygiene)",
    13: "Bad Service",
    14: "Bad Service",
    15: "Quality(Stale/Cold/Hygiene)",
    16: "Bad Service",
    17: "Quality(Stale/Cold/Hygiene)",
    18: "Quality(Stale/Cold/Hygiene)",
}

In [52]:
df_topics = pd.DataFrame(
    {"order_id": df_caption_cleaned["order_id"], "Topic_ID": topics}
)

df_topics["Topic_Name"] = df_topics["Topic_ID"].map(topic_labels)

df_final = df.merge(df_topics, on="order_id", how="left")

print(df_final.head())


df_final.to_excel("quick_bite_bad_reviews_with_topics.xlsx", index=False)

          order_id               review  Topic_ID   Topic_Name
0  ORD202504000898  Not worth the price         9   High Price
1  ORD202504020679  Not worth the price         9   High Price
2  ORD202506000002  Not worth the price         9   High Price
3  ORD202506000005   Average experience        10  Bad Service
4  ORD202506000006            Bad taste         3    Bad Taste
